# Searching

A fundamental area in Computer Science is to develop _algorithmic_ processes for
efficiently _storing_ and _retrieving_ data with good performance. Nowadays,
data is fast-produced leading to high volume of data in no time. It is
anticipated that _Internet of Things_ (IoT) devices will generate [400
zettabytes](https://goo.gl/xu4NcT) of data in 2018. Performing analytics or
science on such large datasets poses many problems with performance. Two
important building blocks in solving such problems are _Sorting_ and _Searching_
algorithms. From the previous lectures, several classes of data structures were
introduced such as _lists_, _trees_ and _graphs_ that maps a given dataset into
the organization of that data structure. On a functional-level, the data is the
same but _working_ with the data structure such as performing an _insert_ or
_finding an element_ comes with it's trade-offs to performance. This can be
important to consider depending on the character of the dataset and it's
application constraints.

The goal of this assignment is to separate the internals of a data structure
from its operations to understand how the overall performance varies between a
sorted and unsorted data structure. The two performance aspects that you will
investigate is the **time complexity** and **space complexity**. The data
structure you will work with is a conventional 2D list structure also known as a
*Matrix*. The concepts and performance measurements discussed in this assignment
should be applicable for other data structures such as a _graph_.

In this assignment, you will work with an open public dataset from [_Brandweer
Amsterdam-Amstelland_](https://goo.gl/tE3Ahi) that contains over **125.000**
fire alarms reported in this region. The dataset is provided in a compressed
zip-format and the resulting CSV-file will be ~124MB. While there are handy
libraries such as [_pandas_](http://pandas.pydata.org/) that can read a CSV-file
into a table, we will only restrict to using core libraries that is part of
Python. To read a CSV-file to a 2D array, we do the following:

In [3]:
import csv #NB: don't forget to include the csv library

def load(csv_file):
    """
        Loads the data from the file in the provided argument into a 2D array
    """
    with open(csv_file) as csvDataFile:
        return [row for row in csv.reader(csvDataFile, delimiter=';')]

If we inspect the first element of the resulting array, we obtain information
about the structure of the dataset and this is shown below. Consecutive elements
of the array follow this format where the first element in the row is the
*incident_id* and the last element is the *gemeente*
```
[['incident_id',
  'start_tijd',
  'incident_type',
  'landelijke_meldingsclassificatie_niveau1',
  'landelijke_meldingsclassificatie_niveau2',
  'landelijke_meldingsclassificatie_niveau3',
  'datum',
  'jaar',
  'maand_nr',
  'maand_naam',
  'dag_nr',
  'dag_naam',
  'week_nr',
  'kwartaal',
  'prioriteit',
  'uur',
  'dagdeel',
  'objecttype',
  'objectfunctie',
  'buurt',
  'wijk',
  'woonplaats',
  'gemeente'],
  ......
  ]
```

## Ordering the dataset
We can observe by inspecting the first 5 rows in the 2D array that the data is
_unordered_, the *incident_id* that represents the unique identification of each
fire alarm event and the creation time stamp of each event is out-of-sync.


**T1 (20 points):** Your first task is to implement the **insertion sort
algorithm** to sort the dataset according to the *incident_id* and the creation
date. Notice that the algorithm is slow and will not scale. Therefore only a
subset of the dataset will be used. From the function definition below, the
_table_ argument represents the 2D array, _f_ is the compare function and _limit_
describes the size of the dataset (NB: the offset is always the first element)

In [99]:
def insertionSort(table, f, limit=150):
    """
        Takes a data table, take a subset of 150 elements and sorts it
        NB: first value of the array is a schema of the data
    """
    subtable = table[1:limit+1]
    #implementation
    sortedTable = []
    
    for row in subtable:
        index = 0
        if (len(sortedTable) == 0):
            sortedTable.append(row)
        
        
        for element in sortedTable:
            if f(element, row):
                sortedTable.insert(index, row)
                break
            index += 1
        
        if index == len(sortedTable):
            sortedTable.append(row)
    
    for i in range(0, len(sortedTable)):
        table[i] = sortedTable[i]

Use `insertionSort` to sort the dataset on incident id and print the first 20 id's. (Use the `cmpf` function to do this)

In [38]:
incidents = load('brwaa_2010-2015.csv')
insertionSort(incidents, cmpf)

for i in range(0, 20):
    print(incidents[i][0])

2986
7056
10427
16747
20505
22992
31068
33939
39832
39925
40547
40663
40911
43049
43247
45831
45965
48323
49539
51754


The compare function _f_ that is provided in the argument list is used for
comparing two elements and is the piece of information the sort algorithm needs
to know in which order to sort elements. The _incident_id_ should be sorted as an
Integer but in the 2D array it is a String. Therefore we need to
convert to an Integer and also explain the _natural order_ of this type. The "0"
indicates incident id

In [14]:
def cmpf(rowA , rowB):
    return int(rowA[0]) > int(rowB[0])

Similarly, this has to be done for date comparison 

In [4]:
import datetime

def cmpfDate(rowA , rowB): 
    #6 = datum
    # 2008/01/03 00:00:00.000000000, remove microsec
    return datetime.datetime.strptime(rowA[6][:-10],'%Y/%m/%d %H:%M:%S') > datetime.datetime.strptime(rowB[6][:-10],'%Y/%m/%d %H:%M:%S')

After implementing the algorithm, try out the following _limit_ values:

- limit=50
- limit=100
- limit=150
- limit=200
- limit=250

Run the algorithm for each of these limit values and measure how long each of them takes.

In [52]:
import time

for i in range(1,6):
    incidents = load('brwaa_2010-2015.csv')
    start_time = time.time()
    insertionSort(incidents, cmpf, i*50)
    print("Limit " + str(i*50) + " " + str(1000*(time.time() - start_time)) + " ms")

Limit 50 1.0001659393310547 ms
Limit 100 1.4998912811279297 ms
Limit 150 4.00233268737793 ms
Limit 200 6.002187728881836 ms
Limit 250 10.005950927734375 ms


Notice that the time increases with the size of the data. An improvement to the
algorithm is to treat this problem using a *divide and conquer*-approach that
divides the dataset into sub-lists. This improvement is called the *shell
algorithm*


**T2 (10 points):** Your task is to modify the current implementation of the
**insertion sort** algorithm to work with *shell sort algorithm*. The modified
version should be called **gapInsertionSort** and use the arguments specified
below. You have to solve how the **startposition** and **sublistcount** are
used to make this work.

In [33]:
def shellSort(table, f, limit):
    subtable = table[1:limit+1]
    sublistcount = len(subtable)//2
    while sublistcount > 0:
        for startposition in range(sublistcount):
            #implement the function call below
            gapInsertionSort(subtable,f,startposition,sublistcount)

        sublistcount = sublistcount // 2
    
    return subtable

In [34]:
def gapInsertionSort(table, f, startposition, sublistcount):
    indices = []
    current_index = startposition
    while current_index < len(table):
        indices.append(current_index)
        current_index += sublistcount
    
    sortedTable = []
    
    for i in indices:
        index = 0

        for element in sortedTable:
            if f(element, table[i]):
                sortedTable.insert(index, table[i])
                break
            index += 1
        
        if index == len(sortedTable):
            sortedTable.append(table[i])
        
    for i in range(len(indices)):
        table[indices[i]] = sortedTable[i]

Use shellSort to sort the first 300 entries in the table on date and print the first 10 incident id's

In [89]:
incidents = load('brwaa_2010-2015.csv')
subtable = shellSort(incidents, cmpfDate, 300)

for i in range(10):
    print(subtable[i][0])

152257
151496
151343
150103
150062
149810
149393
149114
149008
148629


This algorithm is faster than insertion sort, but it is still not fast enough to sort all 124,000 records, so we need to have an algorithm with a lower time complexity. There are two other algorithms that have a "O(n log n)" time complexity but their space complexity varies. For *merge sort*, the space complexity is O (n) which means a linear growth of the space but for a *quick sort*, the space complexity is constant but the performance could degrade to "O(n^2)" depending on the data and selection of the pivot. For instance, if the dataset is ordered or the elements are the same. This is not the case for the dataset that we are using

**T3 (30 points):** Your task is to implement the *quick sort algorithm*, the
stubs for the algorithm is presented here and the challenge here is choosing the
pivot such that it can in short time process all the records.

In [64]:
def quickSort(table,f):
    quickSortHelper(table,f,0,len(table)-1)

def quickSortHelper(table,f,first,last):
    if first<last:

        splitpoint = partition(table,f,first,last)

        quickSortHelper(table,f,first,splitpoint-1)
        quickSortHelper(table,f,splitpoint+1,last)

def partition(table,f,first,last):
    pivotvalue = table[first]

    leftmark = first+1
    rightmark = last

    done = False
    while not done:

        while leftmark <= rightmark and  not f(table[leftmark], pivotvalue):
            leftmark = leftmark + 1

        while not f(pivotvalue, table[rightmark]) and rightmark >= leftmark:
            rightmark = rightmark -1

        if rightmark < leftmark:
            done = True
        else:
            temp = table[leftmark]
            table[leftmark] = table[rightmark]
            table[rightmark] = temp

    temp = table[first]
    table[first] = table[rightmark]
    table[rightmark] = temp

    return rightmark

Use quick sort algorithm to sort the table on date and print the first 10 incident id's

In [96]:
incidents = load('brwaa_2010-2015.csv')
subtable = incidents[1:len(incidents)]

quickSort(subtable, cmpfDate)

print([i[0] for i in subtable][0:10])

['154063', '154060', '154041', '154054', '154045', '154061', '154046', '154048', '154050', '154002']


**T4 (10 points):** Run the shellSort and the quickSort algorithms on the same limit values as in the previous task and measure the run times.

In [109]:
import time

incidents = load('brwaa_2010-2015.csv')
r = range(100, 2000, 100)
print("insertionSort:")
for limit in r:
    temp_incidents = incidents[1:limit+1]
    start_time = time.time()
    insertionSort(temp_incidents, cmpf, limit)
    print("Limit " + str(limit) + " " + str(int(100000*(time.time() - start_time)) / 100) + " ms")

print("Shellsort:")
for limit in r:
    temp_incidents = incidents[1:limit+1]
    start_time = time.time()
    shellSort(temp_incidents, cmpf, limit)
    print("Limit " + str(limit) + " " + str(int(100000*(time.time() - start_time)) / 100) + " ms")
    
print("quickSort:")
for limit in r:
    temp_incidents = incidents[1:limit+1]
    start_time = time.time()
    quickSort(temp_incidents, cmpf)
    print("Limit " + str(limit) + " " + str(int(100000*(time.time() - start_time)) / 100) + " ms")

insertionSort:
Limit 50 0.49 ms
Limit 150 4.0 ms
Limit 250 9.5 ms
Limit 350 18.01 ms
Limit 450 29.52 ms
Limit 550 45.03 ms
Limit 650 61.54 ms
Limit 750 92.57 ms
Limit 850 119.07 ms
Limit 950 145.6 ms
Limit 1050 172.63 ms
Limit 1150 217.65 ms
Limit 1250 236.15 ms
Limit 1350 497.86 ms
Limit 1450 349.24 ms
Limit 1550 381.26 ms
Limit 1650 442.29 ms
Limit 1750 486.88 ms
Limit 1850 550.38 ms
Limit 1950 604.42 ms
Shellsort:
Limit 50 0.99 ms
Limit 150 13.0 ms
Limit 250 33.03 ms
Limit 350 69.53 ms
Limit 450 99.07 ms
Limit 550 196.14 ms
Limit 650 249.17 ms
Limit 750 329.73 ms
Limit 850 383.28 ms
Limit 950 457.8 ms
Limit 1050 691.52 ms
Limit 1150 818.56 ms
Limit 1250 1194.94 ms
Limit 1350 1070.27 ms
Limit 1450 1382.5 ms
Limit 1550 1239.03 ms
Limit 1650 1518.58 ms
Limit 1750 1875.85 ms
Limit 1850 1732.73 ms
Limit 1950 1987.0 ms
quickSort:
Limit 50 0.5 ms
Limit 150 1.0 ms
Limit 250 2.0 ms
Limit 350 3.48 ms
Limit 450 4.0 ms
Limit 550 5.51 ms
Limit 650 6.49 ms
Limit 750 7.52 ms
Limit 850 9.0 ms
Limit

## Searching the dataset
The final part of the assignment is to perform find elements in a data set. We
will use two search techniques, a simple linear search and binary search.

**T5 (20 points):** Create a linear search algorithm and your task is to find the
following id's: 93094, 66515, 47279, 122471, 16515

In [ ]:
def lsearch(table,f,fieldIndex,find):
    pass

Use `lsearch` to find the given id's and print their date (If it can't be found print something like: N/A). Also measure how long it takes combined to find them.

*Hint 1:* You can use the same compare function as in the previous tasks to
provide for *f*

*Hint 2:* The *fieldIndex=...* represents the number of the column in the table for which you want to search, so if you use an index of 0 you will search for an incident id

We could observe that for two of the search queries, they took considerable
time, this due to the fact that we don't know where the items are. Many
algorithms are designed with the idea that the data set is sorted. Therefore, we
will try the same on a sorted data set. This way we might not need to search the entire table to know that a certain value does not appear in it.

In [ ]:
def sortedLSearch(sortedTable,f,fieldIndex,find):
    pass

Again try to search for the same id's and measure how long it takes (don't count the time it takes to sort the table), you will probably notice that the search executed faster. 

We can further enhance the *search process* by exploiting the properties of
using a sorted array. A binary search algorithm uses the properties of a sorted
algorithm. The process is very intuitive and can be illustrated using the
*De Telefoongids* that is a sorted dataset with name and telephone numbers ordered.
If you look for someone with the surname "Hejderup", you will start at the index
H and continue the search there.

**T6 (20 points):** Your task is to implement a binary search version with the
same search queries as in the previous task.

In [ ]:
def bsearch(sortedTable,f,fieldIndex,find):
    pass

Again try to search for the same id's and measure how long it takes (don't count the time it takes to sort the table)

*Optional (Bonus task)*

**T7 (10 points):** Last task is to find multiple entries, your task is to modify
the binary search such that all results for the 'gemeente' of Amsterdam are
returned. 

In [ ]:
def multiplebsearch(sortedTable,f,fieldIndex,find):
    pass

Print all the incident id's of incidents that happened in the 'gemeente' Amsterdam